In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
from six.moves import range
from six.moves.urllib.request import urlretrieve
from IPython.display import display, Image
from scipy import ndimage
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import sys
import tarfile
import h5py


In [3]:
file = open('SVHN_multi.pickle','rb')
data=pickle.load(file)
train_data=data['train_dataset']
train_label=data['train_labels']
valid_data=data['valid_dataset']
valid_label=data['valid_labels']
test_data=data['test_dataset']
test_label=data['test_labels']


print (train_data.shape)
print (train_label.shape)
print (valid_data.shape)
print (valid_label.shape)
print (test_data.shape)
print (test_label.shape)

#print train_data

(230070, 32, 32, 1)
(230070, 6)
(5684, 32, 32, 1)
(5684, 6)
(13068, 32, 32, 1)
(13068, 6)


In [ ]:
# Accuracy Function

In [6]:
def accuracy(predictions, labels):

  return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])
#CNN
#Weight and Bias initialization
pixel =32
labels=11
channels=1
noofimages = 64
stride=5
noffilters1=16
noffilters2=32
hidden=128

shape = [noofimages, pixel, pixel, channels]


graph = tf.Graph()

with graph.as_default():
        #tf.get_variable --shared variable
        tf_train_data = tf.placeholder(tf.float32, shape=(noofimages, pixel, pixel, channels))
        tf_train_label = tf.placeholder(tf.int32, shape=(noofimages, 6))
        tf_valid_data = tf.constant(valid_data)
        tf_test_data = tf.constant(test_data)
        #weight initialization method --xavier
        w1=tf.get_variable("weight1", shape=[stride, stride, channels, noffilters1],\
           initializer=tf.constant_initializer(0))
        #b1=tf.get_variable("bias1",initializer=tf.zeros([noffilters1]))
        b1=tf.Variable(tf.constant(1.0, shape=[noffilters1]), name='B1')
        print ("bias shape")
        print (b1.get_shape())
        w2=tf.get_variable("weight2", shape=[stride, stride, noffilters1, noffilters2],\
          initializer=tf.constant_initializer(0))
        b2=tf.Variable(tf.constant(1.0, shape=[noffilters2]), name='B2')
        w3=tf.get_variable("weight3", shape=[stride, stride, noffilters2, hidden],\
           initializer=tf.constant_initializer(0))
        b3=tf.Variable(tf.constant(1.0, shape=[hidden]), name='B3')
        
        
        nn_w1=tf.get_variable("nn_weight1", shape=[hidden,labels],\
             initializer=tf.constant_initializer(0))
        nn_b1=tf.Variable(tf.constant(1.0, shape=[labels]), name='nnbias1')
        nn_w2=tf.get_variable("nn_weight2", shape=[hidden,labels],\
             initializer=tf.constant_initializer(0))
        nn_b2=tf.Variable(tf.constant(1.0, shape=[labels]), name='nnbias2')
        nn_w3=tf.get_variable("nn_weight3",shape=[hidden,labels],\
             initializer=tf.constant_initializer(0))
        nn_b3=tf.Variable(tf.constant(1.0, shape=[labels]), name='nnbias3')
        nn_w4=tf.get_variable("nn_weight4",shape=[hidden,labels],\
             initializer=tf.constant_initializer(0))
        nn_b4=tf.Variable(tf.constant(1.0, shape=[labels]), name='nnbias4')
        nn_w5=tf.get_variable("nn_weight5",shape=[hidden,labels],\
             initializer=tf.constant_initializer(0))
        nn_b5=tf.Variable(tf.constant(1.0, shape=[labels]), name='nnbias5')
        
        
        def model(data,shape):
            print ("model")
            print ([shape[0], shape[1] * shape[2] * shape[3]])
            layer1 = tf.nn.conv2d(data,w1, [1,1,1,1],padding='VALID', name='cnn1')
            #add bias
            print ("before bias")
            print (layer1.get_shape())
            layer1=layer1+b1
            print ("add bias")
            print (layer1.get_shape())
            #normalize
            layer1=tf.nn.relu(layer1)
            print ("Relu")
            print (layer1.get_shape())
            layer1 = tf.nn.local_response_normalization(layer1)
            #max-pool
            layer1=tf.nn.max_pool(layer1, [1,2,2,1], [1,2,2,1], 'SAME', name='m1')
            
            print ("max pool")
            print (layer1.get_shape())
            
            print (w2.get_shape())
            layer2=tf.nn.conv2d(layer1, w2, [1,1,1,1], padding='VALID', name='cnn2')
            #add bias
            layer2=layer2+b2
            #normalize
            layer2=tf.nn.relu(layer2)
            layer2 = tf.nn.local_response_normalization(layer2)
            #max-pool
            layer2=tf.nn.max_pool(layer2, [1,2,2,1], [1,2,2,1], 'SAME', name='m2')
            
            #print (layer1.get_shape())
            layer3=tf.nn.conv2d(layer2, w3, [1,1,1,1], padding='VALID', name='cnn3')
            #add bia2
            layer3=layer3+b3
            #normalize
            layer3=tf.nn.relu(layer3)
            
           
            #max-pool
            #layer3=tf.nn.max_pool(layer3, [1,2,2,1], [1,2,2,1], 'SAME', name='m3')
            
            #flatten
            layer3 = tf.nn.dropout(layer3, 0.9375)
            print ("Layer3 shape")
            print (layer3.get_shape())
            
            shape = layer3.get_shape().as_list()
            print ([shape[0], shape[1] * shape[2] * shape[3]])
            reshape = tf.reshape(layer3, [shape[0], shape[1] * shape[2] * shape[3]])
            print ("reshape shape")
            print (reshape.get_shape())
            logits1 = tf.matmul(reshape, nn_w1) + nn_b1
            logits2 = tf.matmul(reshape, nn_w2) + nn_b2
            logits3 = tf.matmul(reshape, nn_w3) + nn_b3
            logits4 = tf.matmul(reshape, nn_w4) + nn_b4
            logits5 = tf.matmul(reshape, nn_w5) + nn_b5
            return [logits1, logits2, logits3, logits4, logits5]
        
        [logits1, logits2, logits3, logits4, logits5] = model(tf_train_data, shape)
            #Loss function
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_label[:,1])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_label[:,2])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_label[:,3])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_label[:,4])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, tf_train_label[:,5]))

      # Optimizer.
     #optimizer = tf.train.AdagradOptimizer(0.01).minimize(loss) //static learning rate
        global_step = tf.Variable(0)
    #learning rate with exponential decay.
        learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95) 
        optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

         # Predictions for the training, validation, and test data.
        train_prediction =tf.pack([tf.nn.softmax(model(tf_train_data, shape)[0]),\
                          tf.nn.softmax(model(tf_train_data, shape)[1]),\
                          tf.nn.softmax(model(tf_train_data, shape)[2]),\
                          tf.nn.softmax(model(tf_train_data, shape)[3]),\
                          tf.nn.softmax(model(tf_train_data, shape)[4])])
        valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_data, shape)[0]),\
                          tf.nn.softmax(model(tf_valid_data, shape)[1]),\
                          tf.nn.softmax(model(tf_valid_data, shape)[2]),\
                          tf.nn.softmax(model(tf_valid_data, shape)[3]),\
                          tf.nn.softmax(model(tf_valid_data, shape)[4])])
        test_prediction = tf.pack([tf.nn.softmax(model(tf_test_data, shape)[0]),\
                         tf.nn.softmax(model(tf_test_data, shape)[1]),\
                         tf.nn.softmax(model(tf_test_data,shape)[2]),\
                         tf.nn.softmax(model(tf_test_data,shape)[3]),\
                         tf.nn.softmax(model(tf_test_data,shape)[4])])

        saver = tf.train.Saver()
    
num_steps = 2000

with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()  
        print('Initialized')
        for step in range(num_steps):
                offset = (step * noofimages) % (train_label.shape[0] - noofimages)
                batch_data = train_data[offset:(offset + noofimages), :, :, :]
                batch_labels = train_label[offset:(offset + noofimages),:]
                feed_dict = {tf_train_data : batch_data, tf_train_label : batch_labels}
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)

              
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_label[:,1:6]))
        save_path = saver.save(session, "CNN.ckpt")
        print("Model saved in file: %s" % save_path)

        
    
    
        
        
           

bias shape
(16,)
model
[64, 1024]
before bias
(64, 28, 28, 16)
add bias
(64, 28, 28, 16)
Relu
(64, 28, 28, 16)
max pool
(64, 14, 14, 16)
(5, 5, 16, 32)
Layer3 shape
(64, 1, 1, 128)
[64, 128]
reshape shape
(64, 128)
model
[64, 1024]
before bias
(64, 28, 28, 16)
add bias
(64, 28, 28, 16)
Relu
(64, 28, 28, 16)
max pool
(64, 14, 14, 16)
(5, 5, 16, 32)
Layer3 shape
(64, 1, 1, 128)
[64, 128]
reshape shape
(64, 128)
model
[64, 1024]
before bias
(64, 28, 28, 16)
add bias
(64, 28, 28, 16)
Relu
(64, 28, 28, 16)
max pool
(64, 14, 14, 16)
(5, 5, 16, 32)
Layer3 shape
(64, 1, 1, 128)
[64, 128]
reshape shape
(64, 128)
model
[64, 1024]
before bias
(64, 28, 28, 16)
add bias
(64, 28, 28, 16)
Relu
(64, 28, 28, 16)
max pool
(64, 14, 14, 16)
(5, 5, 16, 32)
Layer3 shape
(64, 1, 1, 128)
[64, 128]
reshape shape
(64, 128)
model
[64, 1024]
before bias
(64, 28, 28, 16)
add bias
(64, 28, 28, 16)
Relu
(64, 28, 28, 16)
max pool
(64, 14, 14, 16)
(5, 5, 16, 32)
Layer3 shape
(64, 1, 1, 128)
[64, 128]
reshape shape
(64

#### 